In [35]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")


In [2]:
df1 = pd.read_csv("dataset/dataset0121.csv")

In [ ]:

df1


In [ ]:
df1.drop('Unnamed: 0',axis=1,inplace=True)
df1

In [ ]:
df1.info()

In [ ]:
df1['State'].unique()

In [7]:
df1.loc[df1['State'] == 'A&N Islands', 'State'] = 'A & N ISLANDS'
df1.loc[df1['State'] == 'D&N Haveli', 'State'] = 'D & N HAVELI'
df1.loc[df1['State'] == 'Delhi UT', 'State'] = 'DELHI'

df1['State'] = pd.Series(str.upper(i) for i in df1['State'])
# df1['DISTRICT'] = pd.Series(str.upper(i) for i in df1['DISTRICT'])

In [ ]:
df1['State'].unique()

In [9]:
states_and_year_crimes = df1

In [ ]:
state_all_crimes_1 = df1.groupby('State').sum()
state_all_crimes_1

In [ ]:
state_and_year_crimes = state_all_crimes_1
state_all_crimes_1.drop('Year',axis=1,inplace=True)
state_all_crimes_1

In [ ]:
col_list_1= list(state_all_crimes_1)                                         
state_all_crimes_1[col_list_1] = state_all_crimes_1[col_list_1].apply(pd.to_numeric, errors='coerce')

# Sum across rows
state_all_crimes_1['Total'] = state_all_crimes_1[col_list_1].sum(axis=1)

all_crimes_1 = state_all_crimes_1

all_crimes_1

In [ ]:
total_df=state_all_crimes_1.sum(axis=0).reset_index()
tf=pd.DataFrame(total_df)
tf

In [ ]:
tf

In [ ]:
import plotly.express as px

sorted_df = state_all_crimes_1.sort_values('Total',ascending=False)
fig = px.bar( x=tf["index"],y=tf[0], color=tf[0], 
             labels={'x': "Crimes", 'y': "Count"}, title="Total Cases", 
             color_continuous_scale='Viridis')
fig.show()


In [16]:
# tf=tf.drop([0])

In [17]:
# tf=tf.drop([8])

In [ ]:
import plotly.express as px

sorted_df = state_all_crimes_1.sort_values('Total',ascending=False)
total_df=sorted_df.sum(axis=0).reset_index()
total_df

In [ ]:
state_all_crimes = df1.groupby('Year').sum()
state_all_crimes

In [ ]:
col_list = list(state_all_crimes)
state_all_crimes[col_list] = state_all_crimes[col_list].apply(pd.to_numeric, errors='coerce')
state_all_crimes['Total'] = state_all_crimes[col_list].sum(axis=1)
all_crimes = state_all_crimes.drop(columns=['State'])
all_crimes

In [ ]:
all_crimes.loc[2011, 'Assault against women'] = np.nan
all_crimes['Assault against women'] = all_crimes['Assault against women'].astype(float)
all_crimes['Assault against women'].interpolate(inplace=True)
all_crimes


In [ ]:
total_df_1=all_crimes.reset_index()
tf_1=pd.DataFrame(total_df_1)
tf_1

In [ ]:
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split

x = tf_1[["Year"]]  
y = tf_1[["Rape", "Kidnap And Assault", "Dowry Deaths", 
        "Assault against women", 
        "Assault against modesty of women", "Domestic violence", 
        "Women Trafficking", "Total"]]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = linear_model.LinearRegression()
model.fit(x_train, y_train)
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)
    
fig, ax = plt.subplots(2, 4, figsize=(15, 10))

for i, crime_type in enumerate(y.columns):
    row = i // 4
    col = i % 4
    ax[row, col].scatter(x_train, y_train[crime_type], color='blue', label='Actual (Train)', alpha=0.6)
    ax[row, col].scatter(x_test, y_test[crime_type], color='green', label='Actual (Test)', alpha=0.6)
    ax[row, col].plot(x_train, y_train_pred[:, i], color='red', linewidth=2, label='Predicted (Train)')
    ax[row, col].plot(x_test, y_test_pred[:, i], color='orange', linewidth=2, label='Predicted (Test)')
    ax[row, col].set_title(crime_type)
    ax[row, col].set_xlabel('Year')
    ax[row, col].set_ylabel('Cases')
    ax[row, col].legend()

plt.tight_layout()  # Adjust subplots to fit into figure area
plt.show()

In [ ]:
model.coef_

In [ ]:
model.intercept_

### performance metrics

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
metrics = {
    'Crime Category': [],
    'MAE (Train)': [],
    'MSE (Train)': [],
    'RMSE (Train)': [],
    'R^2 (Train)': [],
    'MAE (Test)': [],
    'MSE (Test)': [],
    'RMSE (Test)': [],
    'R^2 (Test)': []
}

for i, crime_type in enumerate(y.columns):
    metrics['Crime Category'].append(crime_type)
    
    mae_train = mean_absolute_error(y_train[crime_type], y_train_pred[:, i])
    mse_train = mean_squared_error(y_train[crime_type], y_train_pred[:, i])
    rmse_train = np.sqrt(mse_train) 
    r2_train = r2_score(y_train[crime_type], y_train_pred[:, i])
    
    mae_test = mean_absolute_error(y_test[crime_type], y_test_pred[:, i])
    mse_test = mean_squared_error(y_test[crime_type], y_test_pred[:, i])
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(y_test[crime_type], y_test_pred[:, i])
    
    metrics['MAE (Train)'].append(mae_train)
    metrics['MSE (Train)'].append(mse_train)
    metrics['RMSE (Train)'].append(rmse_train)
    metrics['R^2 (Train)'].append(r2_train)
    metrics['MAE (Test)'].append(mae_test)
    metrics['MSE (Test)'].append(mse_test)
    metrics['RMSE (Test)'].append(rmse_test)
    metrics['R^2 (Test)'].append(r2_test)

metrics_df = pd.DataFrame(metrics)
print(metrics_df)

### Statistical test using randomfoest regressor

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import ttest_rel

rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(x_train, y_train)

y_train_pred_rf = rf_model.predict(x_train)
y_test_pred_rf = rf_model.predict(x_test)


In [ ]:
mse_train_lr = mean_squared_error(y_train, y_train_pred)
mse_test_lr = mean_squared_error(y_test, y_test_pred)

mse_train_rf = mean_squared_error(y_train, y_train_pred_rf)
mse_test_rf = mean_squared_error(y_test, y_test_pred_rf)

print("Linear Regression MSE on Train Data: ", mse_train_lr/10e7,"e+07")
print("Linear Regression MSE on Test Data: ", mse_test_lr/10e7,"e+07")
print("Random Forest MSE on Train Data: ", mse_train_rf/10e7,"e+07")
print("Random Forest MSE on Test Data: ", mse_test_rf/10e7,"e+07")


In [ ]:
import numpy as np

for i, target_name in enumerate(y.columns):
    y_test_pred_target = y_test_pred[:, i]
    y_test_pred_rf_target = y_test_pred_rf[:, i]

    t_stat, p_value = ttest_rel(y_test_pred_target, y_test_pred_rf_target)

    print(f"Results for target: {target_name}")
    print(f"t-statistic: {t_stat}")
    print(f"p-value: {p_value}")

    if p_value < 0.05:
        print("The difference in model performance is statistically significant.\n")
    else:
        print("The difference in model performance is not statistically significant.\n")

### LOOCV

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

loo = LeaveOneOut()
mse_lr_list = []
mse_rf_list = []

for train_index, test_index in loo.split(x):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    lr_model = LinearRegression()
    lr_model.fit(x_train, y_train)
    y_test_pred_lr = lr_model.predict(x_test)
    mse_lr = mean_squared_error(y_test, y_test_pred_lr)
    mse_lr_list.append(mse_lr)
    
    rf_model = RandomForestRegressor(random_state=42)
    rf_model.fit(x_train, y_train)
    y_test_pred_rf = rf_model.predict(x_test)
    mse_rf = mean_squared_error(y_test, y_test_pred_rf)
    mse_rf_list.append(mse_rf)

avg_mse_lr = np.mean(mse_lr_list)
avg_mse_rf = np.mean(mse_rf_list)

print(f"Average MSE for Linear Regression (LOOCV): {avg_mse_lr/10e7}e+07")
print(f"Average MSE for Random Forest (LOOCV): {avg_mse_rf/10e7}e+07")


### K fold cross validation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
mse_lr_list = []
mse_rf_list = []

for train_index, test_index in kf.split(x):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    lr_model = LinearRegression()
    lr_model.fit(x_train, y_train)
    y_test_pred_lr = lr_model.predict(x_test)
    mse_lr = mean_squared_error(y_test, y_test_pred_lr)
    mse_lr_list.append(mse_lr)
    
    rf_model = RandomForestRegressor(random_state=42)
    rf_model.fit(x_train, y_train)
    y_test_pred_rf = rf_model.predict(x_test)
    mse_rf = mean_squared_error(y_test, y_test_pred_rf)
    mse_rf_list.append(mse_rf)

avg_mse_lr = np.mean(mse_lr_list)
avg_mse_rf = np.mean(mse_rf_list)

print(f"Average MSE for Linear Regression: {avg_mse_lr/10e7}e+07")
print(f"Average MSE for Random Forest: {avg_mse_rf/10e7}e+07")


### ARIMA testing till 2021

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split

crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths", 
            "Assault against women", 
            "Assault against modesty of women", "Domestic violence", 
            "Women Trafficking", "Total"]

x = tf_1[["Year"]]  
y = tf_1[crime_types]  

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

fig, ax = plt.subplots(2, 4, figsize=(15, 10))              

for i, crime_type in enumerate(crime_types):
    row = i // 4  
    col = i % 4  
    

    y_train_crime = y_train[crime_type]
    y_test_crime = y_test[crime_type]
    
    model = ARIMA(y_train_crime, order=(1, 1, 1))  
    model_fit = model.fit()


    y_train_pred = model_fit.fittedvalues
    y_test_pred = model_fit.forecast(steps=len(y_test_crime))

    ax[row, col].scatter(x_train, y_train_crime, color='blue', label='Actual (Train)', alpha=0.6)
    ax[row, col].scatter(x_test, y_test_crime, color='green', label='Actual (Test)', alpha=0.6)
    ax[row, col].plot(x_train, y_train_pred, color='red', linewidth=2, label='Predicted (Train)')
    ax[row, col].plot(x_test, y_test_pred, color='orange', linewidth=2, label='Predicted (Test)')
    
    ax[row, col].set_title(crime_type)
    ax[row, col].set_xlabel('Year')
    ax[row, col].set_ylabel('Cases')
    ax[row, col].legend()

plt.tight_layout()  
plt.show()

### CV on ARIMA

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Assuming tf_1 is your dataframe
crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths",
               "Assault against women", "Assault against modesty of women",
               "Domestic violence", "Women Trafficking", "Total"]

def time_series_cv(data, n_splits=5):
    n = len(data)
    k = n // n_splits
    for i in range(n_splits):
        train = data.iloc[:n - (i+1)*k]
        test = data.iloc[n - (i+1)*k:n - i*k]
        yield train, test

mse_scores = {crime_type: [] for crime_type in crime_types}

for crime_type in crime_types:
    y = tf_1[["Year", crime_type]].set_index("Year")
    
    for train, test in time_series_cv(y):
        model = ARIMA(train, order=(1, 1, 1))
        model_fit = model.fit()
        
        forecast = model_fit.forecast(steps=len(test))
        mse = mean_squared_error(test, forecast)
        mse_scores[crime_type].append(mse/10e7)

print("Average MSE scores:")
for crime_type, scores in mse_scores.items():
    avg_mse = np.mean(scores)
    print(f"{crime_type}: {avg_mse:.2f}e+07")

print("\nMSE scores for each fold:")
for crime_type, scores in mse_scores.items():
    print(f"\n{crime_type}:")
    for i, score in enumerate(scores, 1):
        print(f"  Fold {i}: {score:.2f}e+07")   

### ARIMA till 2024

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths", 
               "Assault against women", 
               "Assault against modesty of women", "Domestic violence", 
               "Women Trafficking", "Total"]

fig, ax = plt.subplots(2, 4, figsize=(15, 10))


for i, crime_type in enumerate(crime_types):
    row = i // 4  
    col = i % 4  
    
    y_full = tf_1[crime_type]

    model = ARIMA(y_full, order=(1, 1, 1))  
    model_fit = model.fit()

    forecast_steps = 3
    y_forecast = model_fit.forecast(steps=forecast_steps)

    future_years = pd.DataFrame({
        'Year': [2022, 2023, 2024],
        crime_type: y_forecast
    }).set_index('Year')

    full_data = pd.concat([tf_1[[crime_type]].set_index(tf_1['Year']), future_years])

    ax[row, col].plot(full_data.index, full_data[crime_type], marker='o', label='Historical Data', color='blue')
    ax[row, col].plot(future_years.index, future_years[crime_type], marker='o', color='red', label='Forecasted Data')

    ax[row, col].set_title(crime_type)              
    ax[row, col].set_xlabel('Year')
    ax[row, col].set_ylabel('Cases')
    ax[row, col].legend()

plt.tight_layout() 
plt.show()              

### Support Vector Regressor

In [ ]:
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x = tf_1[["Year"]]  
y = tf_1[["Rape", "Kidnap And Assault", "Dowry Deaths", 
           "Assault against women", 
           "Assault against modesty of women", "Domestic violence", 
           "Women Trafficking", "Total"]]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

fig, ax = plt.subplots(2, 4, figsize=(15, 10))

for i, crime_type in enumerate(y.columns):
    scaler_x = StandardScaler()
    x_train_scaled = scaler_x.fit_transform(x_train)
    x_test_scaled = scaler_x.transform(x_test)
    
    scaler_y = StandardScaler()
    y_train_scaled = scaler_y.fit_transform(y_train[[crime_type]])
    
    model = SVR(kernel='rbf')
    model.fit(x_train_scaled, y_train_scaled.ravel())  # Use ravel to flatten y_train

    # Make predictions
    y_train_pred_scaled = model.predict(x_train_scaled)
    y_test_pred_scaled = model.predict(x_test_scaled)
    
    y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled.reshape(-1, 1))
    y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled.reshape(-1, 1))
    
    row = i // 4
    col = i % 4
    ax[row, col].scatter(x_train, y_train[crime_type], color='blue', label='Actual (Train)', alpha=0.6)
    ax[row, col].scatter(x_test, y_test[crime_type], color='green', label='Actual (Test)', alpha=0.6)
    ax[row, col].plot(x_train, y_train_pred, color='red', linewidth=2, label='Predicted (Train)')
    ax[row, col].plot(x_test, y_test_pred, color='orange', linewidth=2, label='Predicted (Test)')
    ax[row, col].set_title(crime_type)
    ax[row, col].set_xlabel('Year')
    ax[row, col].set_ylabel('Cases')
    ax[row, col].legend()

plt.tight_layout() 
plt.show()


### KN Regression       

In [ ]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
    
x = tf_1[["Year"]]  
y = tf_1[["Rape", "Kidnap And Assault", "Dowry Deaths", 
           "Assault against women", 
           "Assault against modesty of women", "Domestic violence", 
           "Women Trafficking", "Total"]]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

fig, ax = plt.subplots(2, 4, figsize=(15, 10))

for i, crime_type in enumerate(y.columns):
    scaler_x = StandardScaler()
    x_train_scaled = scaler_x.fit_transform(x_train)
    x_test_scaled = scaler_x.transform(x_test)
    
    scaler_y = StandardScaler()
    y_train_scaled = scaler_y.fit_transform(y_train[[crime_type]])
    
    model = KNeighborsRegressor(n_neighbors=5) 
    model.fit(x_train_scaled, y_train_scaled.ravel())  

    y_train_pred_scaled = model.predict(x_train_scaled)
    y_test_pred_scaled = model.predict(x_test_scaled)
    
    y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled.reshape(-1, 1))
    y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled.reshape(-1, 1))
                
    row = i // 4
    col = i % 4
    ax[row, col].scatter(x_train, y_train[crime_type], color='blue', label='Actual (Train)', alpha=0.6)
    ax[row, col].scatter(x_test, y_test[crime_type], color='green', label='Actual (Test)', alpha=0.6)
    ax[row, col].plot(x_train, y_train_pred, color='red', linewidth=2, label='Predicted (Train)')
    ax[row, col].plot(x_test, y_test_pred, color='orange', linewidth=2, label='Predicted (Test)')
    ax[row, col].set_title(crime_type)
    ax[row, col].set_xlabel('Year')
    ax[row, col].set_ylabel('Cases')
    ax[row, col].legend()

plt.tight_layout()  
plt.show()



### Decision trees

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

# List of crime types
crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths", 
               "Assault against women", 
               "Assault against modesty of women", "Domestic violence", 
               "Women Trafficking", "Total"]

# Extract features and targets
x = tf_1[["Year"]]  
y = tf_1[crime_types]  

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

# Create the plot
fig, ax = plt.subplots(2, 4, figsize=(15, 10))              

for i, crime_type in enumerate(crime_types):
    row = i // 4  
    col = i % 4  

    # Get the training and testing data for the current crime type
    y_train_crime = y_train[crime_type]
    y_test_crime = y_test[crime_type]
    
    # Train Decision Tree Regressor model
    model = DecisionTreeRegressor(random_state=42)
    model.fit(x_train, y_train_crime)

    # Predict on training and testing sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Plot the actual vs predicted values
    ax[row, col].scatter(x_train, y_train_crime, color='blue', label='Actual (Train)', alpha=0.6)
    ax[row, col].scatter(x_test, y_test_crime, color='green', label='Actual (Test)', alpha=0.6)
    ax[row, col].plot(x_train, y_train_pred, color='red', linewidth=2, label='Predicted (Train)')
    ax[row, col].plot(x_test, y_test_pred, color='orange', linewidth=2, label='Predicted (Test)')
    
    # Set plot titles and labels
    ax[row, col].set_title(crime_type)
    ax[row, col].set_xlabel('Year')
    ax[row, col].set_ylabel('Cases')
    ax[row, col].legend()

# Adjust layout and show the plot
plt.tight_layout()  
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import mean_squared_error

# List of crime types
crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths", 
               "Assault against women", 
               "Assault against modesty of women", "Domestic violence", 
               "Women Trafficking", "Total"]

# Extract features and targets
x = tf_1[["Year"]]  
y = tf_1[crime_types]

# Set up K-Fold Cross Validation (e.g., 5 folds)
kf = KFold(n_splits=5, shuffle=False)
# Dictionary to store average MSE for each crime type
avg_mse = {}

# Loop through each crime type
for i, crime_type in enumerate(crime_types):
    row = i // 4  
    col = i % 4  

    # Get the data for the current crime type
    y_crime = y[crime_type]

    # Initialize Decision Tree Regressor model
    model = DecisionTreeRegressor(random_state=42)

    # Perform K-Fold Cross-Validation
    y_pred = cross_val_predict(model, x, y_crime, cv=kf)

    # Calculate MSE for each fold
    mse = mean_squared_error(y_crime, y_pred)
    avg_mse[crime_type] = mse
    
# Print the average MSE for each crime type
for crime_type, mse_value in avg_mse.items():
    print(f"Average MSE for {crime_type}: {mse_value/10e7:.4f}")


### Decision trees with lag

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Feature engineering for time series
def create_features(df, crime_types, lags=3, rolling_window=3):
    df_features = df.copy()
    
    # Create lag features
    for crime_type in crime_types:
        for lag in range(1, lags+1):
            df_features[f"{crime_type}_lag_{lag}"] = df[crime_type].shift(lag)

    # Create rolling statistics features (rolling mean and std)
    for crime_type in crime_types:
        df_features[f"{crime_type}_rolling_mean"] = df[crime_type].rolling(window=rolling_window).mean()
        df_features[f"{crime_type}_rolling_std"] = df[crime_type].rolling(window=rolling_window).std()
    
    # Time-based features
    df_features['Year'] = df['Year']
    df_features['Year_diff'] = df['Year'].diff()

    # Drop any rows with NaN values created by lagging/rolling
    df_features = df_features.dropna().reset_index(drop=True)
    
    return df_features

# Create Fourier features for seasonality (e.g., yearly seasonality)
def create_fourier_features(df, period, order):
    t = np.arange(len(df))
    for i in range(1, order + 1):
        df[f'sin_{i}'] = np.sin(2 * np.pi * i * t / period)
        df[f'cos_{i}'] = np.cos(2 * np.pi * i * t / period)
    return df

# Assuming 'tf_1' is your original DataFrame
crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths", 
               "Assault against women", 
               "Assault against modesty of women", "Domestic violence", 
               "Women Trafficking", "Total"]

# Create lagged and rolling features
tf_1_features = create_features(tf_1, crime_types)

# Add Fourier terms for yearly seasonality (period = 1 year, order = 2 or 3)
tf_1_features = create_fourier_features(tf_1_features, period=21, order=3)  # Period = 21 because it's yearly data from 2001-2021

# Separate features (x) and targets (y)
x = tf_1_features.drop(columns=crime_types)  # All columns except crime types are features
y = tf_1_features[crime_types]  # Crime types as targets

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

# Plotting
fig, ax = plt.subplots(2, 4, figsize=(15, 10))              

for i, crime_type in enumerate(crime_types):
    row = i // 4  
    col = i % 4  

    # Get the training and testing data for the current crime type
    y_train_crime = y_train[crime_type]
    y_test_crime = y_test[crime_type]

    # Train Decision Tree Regressor model
    model = DecisionTreeRegressor(random_state=42)
    model.fit(x_train, y_train_crime)

    # Predict on training and testing sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Plot the actual vs predicted values
    ax[row, col].scatter(x_train['Year'], y_train_crime, color='blue', label='Actual (Train)', alpha=0.6)
    ax[row, col].scatter(x_test['Year'], y_test_crime, color='green', label='Actual (Test)', alpha=0.6)
    ax[row, col].plot(x_train['Year'], y_train_pred, color='red', linewidth=2, label='Predicted (Train)')
    ax[row, col].plot(x_test['Year'], y_test_pred, color='orange', linewidth=2, label='Predicted (Test)')
    
    # Set plot titles and labels
    ax[row, col].set_title(crime_type)
    ax[row, col].set_xlabel('Year')
    ax[row, col].set_ylabel('Cases')
    ax[row, col].legend()

# Adjust layout and show the plot
plt.tight_layout()  
plt.show()


### ARIMAX testing till 2021

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split

# List of crime types
crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths", 
               "Assault against women", "Assault against modesty of women", 
               "Domestic violence", "Women Trafficking", "Total"]

# Extract year as the feature
x = tf_1[["Year"]]

# Target variable (the crime data)
y = tf_1[crime_types]

# Let's assume "Total" crime will be used as the exogenous variable (can be adjusted as needed)
exogenous_variables = tf_1[["Rape"]]

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)
exog_train, exog_test = train_test_split(exogenous_variables, test_size=0.2, random_state=42, shuffle=False)

# Set up the plot
fig, ax = plt.subplots(2, 4, figsize=(15, 10))

# Loop through each crime type and fit the ARIMAX model
for i, crime_type in enumerate(crime_types):
    row = i // 4  # Determines the row for the subplot
    col = i % 4   # Determines the column for the subplot
    
    # Train and test data for the specific crime type
    y_train_crime = y_train[crime_type]
    y_test_crime = y_test[crime_type]

    # Fit ARIMAX model with the 'Total' crime as an exogenous variable
    model = ARIMA(y_train_crime, order=(1, 1, 1), exog=exog_train)  # ARIMAX with exogenous variables
    model_fit = model.fit()

    # Predictions
    y_train_pred = model_fit.fittedvalues
    y_test_pred = model_fit.forecast(steps=len(y_test_crime), exog=exog_test)

    # Plotting the results
    ax[row, col].scatter(x_train, y_train_crime, color='blue', label='Actual (Train)', alpha=0.6)
    ax[row, col].scatter(x_test, y_test_crime, color='green', label='Actual (Test)', alpha=0.6)
    ax[row, col].plot(x_train, y_train_pred, color='red', linewidth=2, label='Predicted (Train)')
    ax[row, col].plot(x_test, y_test_pred, color='orange', linewidth=2, label='Predicted (Test)')
    
    # Setting titles and labels
    ax[row, col].set_title(crime_type)
    ax[row, col].set_xlabel('Year')
    ax[row, col].set_ylabel('Cases')
    ax[row, col].legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

# List of crime types
crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths", 
               "Assault against women", "Assault against modesty of women", 
               "Domestic violence", "Women Trafficking", "Total"]

# Extract year as the feature
x = tf_1[["Year"]]

# Target variable (the crime data)
y = tf_1[crime_types]

# Let's assume "Rape" will be used as the exogenous variable (can be adjusted as needed)
exogenous_variables = tf_1[["Total"]]

# Set up TimeSeriesSplit for K-Fold cross-validation (e.g., 5 folds)
tscv = TimeSeriesSplit(n_splits=5)

# Initialize a dictionary to store the average MSE for each crime type
avg_mse = {crime_type: [] for crime_type in crime_types}

# Perform TimeSeries K-Fold Cross Validation
for crime_type in crime_types:
    mse_list = []
    
    # Iterate through the splits
    for train_index, test_index in tscv.split(x):
        # Split the data into training and testing sets for the current fold
        y_train_fold, y_test_fold = y[crime_type].iloc[train_index], y[crime_type].iloc[test_index]
        exog_train_fold, exog_test_fold = exogenous_variables.iloc[train_index], exogenous_variables.iloc[test_index]
        
        # Fit ARIMAX model with the exogenous variable
        model = ARIMA(y_train_fold, order=(1, 1, 1), exog=exog_train_fold)
        model_fit = model.fit()
        
        # Predict on the test set of the current fold
        y_test_pred_fold = model_fit.forecast(steps=len(y_test_fold), exog=exog_test_fold)
        
        # Compute MSE for the current fold
        mse = mean_squared_error(y_test_fold, y_test_pred_fold)
        mse_list.append(mse)
    
    # Store the average MSE for the current crime type
    avg_mse[crime_type] = np.mean(mse_list)

# Print the average MSE for each crime type
for crime_type, mse_value in avg_mse.items():   
    print(f"Average MSE for {crime_type}: {mse_value/10e7:.4f}")


In [46]:
df1 = states_and_year_crimes

crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths", 
            "Assault against women", 
            "Assault against modesty of women", "Domestic violence", 
            "Women Trafficking"]


from sklearn.preprocessing import OneHotEncoder, StandardScaler

# One-hot encode the State column
onehotencoder = OneHotEncoder()
state_encoded = onehotencoder.fit_transform(df1[['State']]).toarray()

# Standardize the crime data (excluding 'Year' column)
scaler = StandardScaler()
crime_data = scaler.fit_transform(df1[crime_types])

# Concatenate one-hot encoded states, year, and scaled crime data
X = np.concatenate([state_encoded, df1[['Year']].values, crime_data], axis=1)
y = df1[crime_types]  # If you're predicting crime rates, use this as target variable


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Regressor for each crime type
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Calculate error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: ", mse)


In [ ]:
# Define high-risk states based on a threshold
df1['High_Risk'] = (df1['Rape'] > df1['Rape'].mean()) & (df1['Domestic violence'] > df1['Domestic violence'].mean())

# Perform train-test split and train a classifier
from sklearn.ensemble import RandomForestClassifier
y_classification = df1['High_Risk'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y_classification, test_size=0.2, random_state=42)
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

# Predict
y_pred = classifier.predict(X_test)

# Check accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Iterate through each state and fit an ARIMA model
states = df1['State'].unique()
for state in states:
    state_data = df1[df1['State'] == state]
    for crime in crime_types:
        model = ARIMA(state_data[crime], order=(1, 1, 1))
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=5)  # Forecasting for next 5 years
        print(f"Forecast for {crime} in {state}: ", forecast)


In [ ]:
import seaborn as sns

# Calculate correlation matrix
correlation_matrix = df1[crime_types].corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation between Crime Types')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split

# List of crime types
crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths", 
               "Assault against women", 
               "Assault against modesty of women", "Domestic violence", 
               "Women Trafficking"]

# Convert "Year" column to datetime
df1['Year'] = pd.to_datetime(df1['Year'], format='%Y')

# Get unique states
states = df1['State'].unique()

# Iterate over each state
for state in states:
    state_data = df1[df1["State"] == state]  # Filter data by state

    # Use 'Year' as index for time series modeling
    state_data = state_data.set_index('Year')
    
    # Plot setup
    fig, ax = plt.subplots(2, 4, figsize=(15, 10))  
    fig.suptitle(f"Crime Analysis for {state}", fontsize=16)

    for i, crime_type in enumerate(crime_types):
        row = i // 4  
        col = i % 4  

        # Train-test split based on the year
        y = state_data[crime_type]
        y_train, y_test = y[:int(len(y) * 0.8)], y[int(len(y) * 0.8):]

        # Fit ARIMA model
        model = ARIMA(y_train, order=(1, 1, 1))  
        model_fit = model.fit()

        # Forecast
        y_train_pred = model_fit.fittedvalues
        y_test_pred = model_fit.forecast(steps=len(y_test))

        # Plot actual vs predicted values
        ax[row, col].scatter(y_train.index, y_train, color='blue', label='Actual (Train)', alpha=0.6)
        ax[row, col].scatter(y_test.index, y_test, color='green', label='Actual (Test)', alpha=0.6)
        ax[row, col].plot(y_train.index, y_train_pred, color='red', linewidth=2, label='Predicted (Train)')
        ax[row, col].plot(y_test.index, y_test_pred, color='orange', linewidth=2, label='Predicted (Test)')

        ax[row, col].set_title(crime_type)
        ax[row, col].set_xlabel('Year')
        ax[row, col].set_ylabel('Cases')
        ax[row, col].legend()

    plt.tight_layout()
    plt.subplots_adjust(top=0.88)  # Adjust for title space
    plt.show()


### kfold on decision trees

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Feature engineering for time series
def create_features(df, crime_types, lags=3, rolling_window=3):
    df_features = df.copy()
    
    # Create lag features
    for crime_type in crime_types:
        for lag in range(1, lags+1):
            df_features[f"{crime_type}_lag_{lag}"] = df[crime_type].shift(lag)

    # Create rolling statistics features (rolling mean and std)
    for crime_type in crime_types:
        df_features[f"{crime_type}_rolling_mean"] = df[crime_type].rolling(window=rolling_window).mean()
        df_features[f"{crime_type}_rolling_std"] = df[crime_type].rolling(window=rolling_window).std()
    
    # Time-based features
    df_features['Year'] = df['Year']
    df_features['Year_diff'] = df['Year'].diff()

    # Drop any rows with NaN values created by lagging/rolling
    df_features = df_features.dropna().reset_index(drop=True)
    
    return df_features

# Create Fourier features for seasonality (e.g., yearly seasonality)
def create_fourier_features(df, period, order):
    t = np.arange(len(df))
    for i in range(1, order + 1):
        df[f'sin_{i}'] = np.sin(2 * np.pi * i * t / period)
        df[f'cos_{i}'] = np.cos(2 * np.pi * i * t / period)
    return df

# Assuming 'tf_1' is your original DataFrame
crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths", 
               "Assault against women", 
               "Assault against modesty of women", "Domestic violence", 
               "Women Trafficking", "Total"]

# Create lagged and rolling features
tf_1_features = create_features(tf_1, crime_types)

# Add Fourier terms for yearly seasonality (period = 1 year, order = 2 or 3)
tf_1_features = create_fourier_features(tf_1_features, period=21, order=3)  # Period = 21 because it's yearly data from 2001-2021

# Separate features (x) and targets (y)
x = tf_1_features.drop(columns=crime_types)  # All columns except crime types are features
y = tf_1_features[crime_types]  # Crime types as targets

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

# Plotting
fig, ax = plt.subplots(2, 4, figsize=(15, 10))              

for i, crime_type in enumerate(crime_types):
    row = i // 4  
    col = i % 4  

    # Get the training and testing data for the current crime type
    y_train_crime = y_train[crime_type]
    y_test_crime = y_test[crime_type]

    # Train Decision Tree Regressor model
    model = DecisionTreeRegressor(random_state=42)
    model.fit(x_train, y_train_crime)

    # Predict on training and testing sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Plot the actual vs predicted values
    ax[row, col].scatter(x_train['Year'], y_train_crime, color='blue', label='Actual (Train)', alpha=0.6)
    ax[row, col].scatter(x_test['Year'], y_test_crime, color='green', label='Actual (Test)', alpha=0.6)
    ax[row, col].plot(x_train['Year'], y_train_pred, color='red', linewidth=2, label='Predicted (Train)')
    ax[row, col].plot(x_test['Year'], y_test_pred, color='orange', linewidth=2, label='Predicted (Test)')
    
    # Set plot titles and labels
    ax[row, col].set_title(crime_type)
    ax[row, col].set_xlabel('Year')
    ax[row, col].set_ylabel('Cases')
    ax[row, col].legend()

# Adjust layout and show the plot
plt.tight_layout()  
plt.show()


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths", 
            "Assault against women", 
            "Assault against modesty of women", "Domestic violence", 
            "Women Trafficking"]
# Define number of folds for K-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=False)

# Initialize a dictionary to store the average MSE for each crime type
avg_mse = {crime_type: [] for crime_type in crime_types}

# Perform K-Fold Cross Validation
for crime_type in crime_types:
    mse_list = []
    
    # Iterate through the folds
    for train_index, test_index in kfold.split(x):
        # Split the data into training and testing sets for the current fold
        x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
        y_train_fold, y_test_fold = y[crime_type].iloc[train_index], y[crime_type].iloc[test_index]
        
        # Train Decision Tree Regressor model on current fold
        model = DecisionTreeRegressor(random_state=42)
        model.fit(x_train_fold, y_train_fold)
        
        # Predict on the test set of the current fold
        y_test_pred_fold = model.predict(x_test_fold)
        
        # Compute MSE for the current fold
        mse = mean_squared_error(y_test_fold, y_test_pred_fold)
        mse_list.append(mse)
    
    # Store the average MSE for the current crime type
    avg_mse[crime_type] = np.mean(mse_list)

# Print the average MSE for each crime type
for crime_type, mse_value in avg_mse.items():
    print(f"Average MSE for {crime_type}: {mse_value/10e7:.4f}")


In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Assuming tf_1 is your dataframe
crime_types = ["Rape", "Kidnap And Assault", "Dowry Deaths",
               "Assault against women", "Assault against modesty of women",
               "Domestic violence", "Women Trafficking", "Total"]

def time_series_cv(data, n_splits=5):
    n = len(data)
    k = n // n_splits
    for i in range(n_splits):
        train = data.iloc[:n - (i+1)*k]
        test = data.iloc[n - (i+1)*k:n - i*k]
        yield train, test

mse_scores = {crime_type: [] for crime_type in crime_types}

for crime_type in crime_types:
    y = tf_1[["Year", crime_type]].set_index("Year")
    
    for train, test in time_series_cv(y):
        model = ARIMA(train, order=(1, 1, 1))
        model_fit = model.fit()
        
        forecast = model_fit.forecast(steps=len(test))
        mse = mean_squared_error(test, forecast)
        mse_scores[crime_type].append(mse/10e7)

print("Average MSE scores:")
for crime_type, scores in mse_scores.items():
    avg_mse = np.mean(scores)
    print(f"{crime_type}: {avg_mse:.2f}e+07")

print("\nMSE scores for each fold:")
for crime_type, scores in mse_scores.items():
    print(f"\n{crime_type}:")
    for i, score in enumerate(scores, 1):
        print(f"  Fold {i}: {score:.2f}e+07")   